# Adversarial EXEmples: evasion attacks against Windows malware detectors

In this laboratory, you will learn how to use SecML to create adversarial examples against Windows malware detector implemented through machine learning techniques. To do so, we will use [SecML malware](https://github.com/pralab/secml_malware), a SecML plugin containing most of the strategies developed to evade detectors.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zangobot/teaching_material/blob/main/03-AdversarialEXEmples.ipynb)

In [ ]:
try:
    import secml_malware
except ImportError:
    %pip install git+https://github.com/elastic/ember.git
    %pip install git+https://github.com/pralab/secml@py311_update
    %pip install secml-malware

# The Windows PE file format
Before starting the explanations of attacks, we remind how Windows programs are stored as file, following the [Windows Portable Executable (PE)](https://learn.microsoft.com/en-us/windows/win32/debug/pe-format) file format.
There are tons of Python libraries for dissecting programs, one of the best is [lief](https://github.com/lief-project/LIEF).
The latter is also used inside `secml-malware` to perturb samples, as shown later on in this tutorial.
Opening an executable is straight-forward:

In [ ]:
import lief

exe_path = 'exe/calc.exe'
exe_object: lief.PE = lief.parse(exe_path)

Now, the `exe_object` contains all the information of the loaded program.
We can look for all the components. For instance, here is how you can read the header metadata:

In [ ]:
print('DOS Header')
print(exe_object.dos_header)

print('PE Header')
print(exe_object.header)

print('Optional Header')
print(exe_object.optional_header)

print('Sections')
for s in exe_object.sections:
    print(s.name, s.characteristics_lists)

This library is also very useful for manipulating the EXEs.
For instance, in few lines of code you can add sections to a program.

In [ ]:
# Name your new section. Size constraint: up to 8 bytes at maximum!
new_section : lief.PE.Section = lief.PE.Section()
new_section.name = '.newsec'
new_section.content = [ord(i) for i in "This is my newly created section"]
new_section.characteristics = lief.PE.SECTION_CHARACTERISTICS.MEM_DISCARDABLE
exe_object.add_section(new_section)

# New section in place! Now we use lief to rebuild the binary.
builder = lief.PE.Builder(exe_object)
builder.build()
exe_object = lief.PE.parse(builder.get_build())
print('Sections')
for s in exe_object.sections:
    print(s.name, s.characteristics_lists)
builder.write('new_exe.file')

As you can see, the new section appeared as last one.
More information on how to use lief on the [documentation of the library](https://lief-project.github.io/doc/stable/index.html).

# Evasion of End-to-end Deep Neural Network for Malware Detection

In this tutorial, you will learn how to use this plugin to test the already implemented attacks against a PyTorch network of your choice.
We first instantiate a deep neural network trained on raw bytes, called MalConv, and we pass it to SecML Malware.

In [ ]:
import os
import magic
from secml.array import CArray

from secml_malware.models.malconv import MalConv
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware, End2EndModel

net = MalConv()
net = CClassifierEnd2EndMalware(net)
net.load_pretrained_model()

print(net)

Firstly, we have created the network (MalConv) and it has been passed wrapped with a *CClassifierEnd2EndMalware* model class.
This object generalizes PyTorch end-to-end ML models.
Since MalConv is already coded inside the plugin, the weights are also stored, and they can be retrieved with the *load_pretrained_model* method.

If you wish to use diffierent weights, pass the path to the PyTorch *pth* file to that method.

In [ ]:
from secml_malware.attack.whitebox.c_header_evasion import CHeaderEvasion

partial_dos = CHeaderEvasion(net, random_init=False, iterations=10, optimize_all_dos=False, threshold=0)

This is how an attack is created, no further action is needed.
The `random_init` parameter specifies if the bytes should be assigned with random values before beginning the optimization process, `iterations` sets the number of steps of the attack, `optimize_all_dos` sets if all the DOS header should be perturbed, or just the first 58 bytes, while `threshold` is the detection threshold used as a stopping condition.

If you want to see how much the network is deteriorated by the attack, set this parameter to 0, or it will stop as soon as the confidence decreases below such value.

In [ ]:
folder = "exe"
X = []
y = []
file_names = []
for i, f in enumerate(os.listdir(folder)):
    path = os.path.join(folder, f)
    if not os.path.isfile(path):
        continue
    if "PE32" not in magic.from_file(path):
        continue
    with open(path, "rb") as file_handle:
        code = file_handle.read()
    x = End2EndModel.bytes_to_numpy(
        code, net.get_input_max_length(), 256, False
    )
    _, confidence = net.predict(CArray(x), True)

    if confidence[0, 1].item() < 0.5:
        continue

    print(f"> Added {f} with confidence {confidence[0,1].item()}")
    X.append(x)
    conf = confidence[1][0].item()
    y.append([1 - conf, conf])
    file_names.append(path)

We load a simple dataset from the provided `exe` that you have filled with malware to test the attacks.
Since we are dealing with SecML, we need to load all the programs inside as CArrays.
Once we have built the dataset, we can run the instantiated attack.

In [ ]:
for sample, label in zip(X, y):
    y_pred, adv_score, adv_ds, f_obj = partial_dos.run(CArray(sample), CArray(label[1]))
    print(partial_dos.confidences_)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(partial_dos.confidences_)

Inside the `adv_ds` object, you can find the adversarial example computed by the attack.
You can reconstruct the functioning example by using a specific function inside the plugin.

In [ ]:
adv_x = adv_ds.X[0,:]
real_adv_x = partial_dos.create_real_sample_from_adv(file_names[0], adv_x)
print(len(real_adv_x))
real_x = End2EndModel.bytes_to_numpy(real_adv_x, net.get_input_max_length(), 256, False)
_, confidence = net.predict(CArray(real_x), True)
print(confidence[0,1].item())

... and you're done!
If you want to create a real sample (stored on disk), just have a look at the `create_real_sample_from_adv` of each attack. It accepts a third string argument that will be used as a destination file path for storing the adversarial example.

We used one attack, which is the Partial DOS one. But what if we want to use others?
SecML malware is shipped with different attacking strategies, that can be easily created similarly to the DOS Header attack. A complete list of them can be found in the [source code](https://github.com/pralab/secml_malware/tree/master/secml_malware/attack/whitebox) or the [documentation](https://secml-malware.readthedocs.io/en/docs/source/secml_malware.attack.whitebox.html) of the other white box attacks.
To give you a practical example, we now use an attack that append bytes at the end of the file (padding) and replace unusued space between sections (slack), and it leverages an iterative version of [FGSM attack](https://arxiv.org/abs/1802.04528) to optimize the byte values of executables.

In [ ]:
from secml_malware.attack.whitebox import CKreukEvasion

ifgsm = CKreukEvasion(net, how_many_padding_bytes=58, epsilon=1.0, iterations=10, threshold=0)
for i, (sample, label) in enumerate(zip(X, y)):
    y_pred, adv_score, adv_ds, f_obj = ifgsm.run(CArray(sample), CArray(label[1]))
    print(ifgsm.confidences_)
    real_adv_x = ifgsm.create_real_sample_from_adv(file_names[i], adv_ds.X[i, :])
    with open(file_names[i], 'rb') as f:
        print('Original length: ', len(f.read()))
    print('Adversarial sample length: ', len(real_adv_x))


In [ ]:
from secml.figure import CFigure

fig = CFigure()
fig.sp.plot(partial_dos.confidences_, label='dos')
fig.sp.plot(ifgsm.confidences_, label='kreuk')
fig.sp.plot([0, 10], [0.5, 0.5], label='threshold', linestyle='--', c='r')
fig.sp.xlabel('Iterations')
fig.sp.ylabel('Malware class confidence')
fig.sp.legend()
fig.show()

As you notice, the attacks behave differently. Can you guess why?

# Gradient-free evasion of Windows Malware Detectors

We move to gradient-free approaches, and we will leverage optimisation of adversarial content injected through new sections inside the input executable.

In [ ]:
import os

import magic
import numpy as np
from secml.array import CArray

from secml_malware.attack.blackbox.c_wrapper_phi import CEnd2EndWrapperPhi
from secml_malware.attack.blackbox.ga.c_base_genetic_engine import CGeneticAlgorithm
from secml_malware.models.c_classifier_end2end_malware import CClassifierEnd2EndMalware
from secml_malware.models.malconv import MalConv

net = CClassifierEnd2EndMalware(MalConv())
net.load_pretrained_model()
net = CEnd2EndWrapperPhi(net)

Firstly, we have created the network (MalConv) and it has been passed wrapped with a *CClassifierEnd2EndMalware* model class.
Since MalConv is already coded inside the plugin, the weights are also stored, and they can be retrieved with the *load_pretrained_model* method.

If you wish to use diffierent weights, pass the path to the PyTorch *pth* file to that method.

Then, we wrap it inside a `CEnd2EndWrapperPhi`, that is an interface that abstracts from the feature extraction phase of the model.
This is needed for the black-box settings of the attack.

We will start by optimizing byte-per-byte through the injection of padding bytes.

In [ ]:

import numpy as np
folder = 'exe'
X_gf = []
y_gf = []
file_names_gf = []
for i, f in enumerate(os.listdir(folder)):
    path = os.path.join(folder, f)
    if not os.path.isfile(path):
        continue
    if "PE32" not in magic.from_file(path):
        continue
    with open(path, "rb") as file_handle:
        code = file_handle.read()
    x = CArray(np.frombuffer(code, dtype=np.uint8))
    _, confidence = net.predict(x, True)

    if confidence[0, 1].item() < 0.5:
        continue

    print(f"> Added {f} with confidence {confidence[0,1].item()}")
    X_gf.append(x)
    y_gf.append(1)
    file_names_gf.append(path)

In [ ]:
from secml_malware.attack.blackbox.c_black_box_padding_evasion import CBlackBoxPaddingEvasionProblem
from secml_malware.attack.blackbox.ga.c_base_genetic_engine import CGeneticAlgorithm
attack_padding = CBlackBoxPaddingEvasionProblem(net, population_size=10, penalty_regularizer=1e-6, iterations=500, how_many_padding_bytes=4096)
engine = CGeneticAlgorithm(attack_padding)
for sample, label in zip(X_gf, y_gf):
    y_pred, adv_score, adv_ds, f_obj = engine.run(sample, label)
    print(engine.confidences_)

Under the hood, the attack is optimizing each the 4096 bytes, one by one. To speed up the process, we can inject chunks of goodware content, and let the optimizer decide how much to extract.

In [ ]:
from secml_malware.attack.blackbox.c_gamma_sections_evasion import CGammaSectionsEvasionProblem
goodware_folder = 'exe/goodware'
section_population, what_from_who = CGammaSectionsEvasionProblem.create_section_population_from_folder(goodware_folder, how_many=10, sections_to_extract=['.rdata'])

gamma_attack = CGammaSectionsEvasionProblem(section_population, net, population_size=10, penalty_regularizer=1e-6, iterations=100, threshold=0)

For the section injection attack implemented with GAMMA, we need first to extract the goodware sections.
Then, we create a `CGammaSectionsEvasionProblem` object, that contains the attack.

In [ ]:
engine_gamma = CGeneticAlgorithm(gamma_attack)
for sample, label in zip(X, y):
    y_pred, adv_score, adv_ds, f_obj = engine_gamma.run(sample, CArray(label[1]))

Inside the `adv_ds` object, you can find the adversarial example computed by the attack.
You can reconstruct the functioning example by using a specific function inside the plugin:

In [ ]:
adv_x = adv_ds.X[0,:]
engine_gamma.write_adv_to_file(adv_x,'adv_exe')
with open('adv_exe', 'rb') as h:
    code = h.read()
real_adv_x = CArray(np.frombuffer(code, dtype=np.uint8))
_, confidence = net.predict(CArray(real_adv_x), True)
print(confidence[0,1].item())

... and you're done!
If you want to create a real sample (stored on disk), just have a look at the `create_real_sample_from_adv` of each attack. It accepts a third string argument that will be used as a destination file path for storing the adversarial example.